# 형태소 분석
- 형태소를 비롯하여, 어근, 접두사/접미사, 품사 등 다양한 언어적 속성의 구조를 파악
- 형태소는 언어학에서 일정한 의미가 있는 가장 작은 말의 단위
- 예)
    - 우리는 한국인 이다 -> 우리(명사), 는(조사), 한국인(명사), 이(동사), 다(어미)


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [2]:
DATA_PATH = os.getcwd()
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device,DATA_PATH

('cuda', 'c:\\study\\04_NLP')

In [3]:
train = pd.read_csv(f"{DATA_PATH}/data/news/train_news.csv")
test = pd.read_csv(f"{DATA_PATH}/data/news/test_news.csv")
train.shape, test.shape

((89320, 3), (38280, 2))

# nltk
- python에서 가장 오래되고 유명한 자연어 처리 라이브러리(한국어 미지원)

In [10]:
import nltk

nltk.download("punkt") # 토큰화 하기 위해 다운로드
nltk.download("stopwords") # 불용어 리스트
nltk.download("averaged_perceptron_tagger") # 품사정보를 추출하기 위해 다운로드

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hopio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hopio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hopio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 토큰화

In [11]:
text = train["desc"][0]
text

'Sudan has decided to postpone a decision to expel the heads of two British aid agencies - Oxfam and Save the Children - citing administrative difficulties and humanitarian grounds.'

In [12]:
from nltk.tokenize import word_tokenize

In [13]:
word_tokenize(text) # 소문자화 X

['Sudan',
 'has',
 'decided',
 'to',
 'postpone',
 'a',
 'decision',
 'to',
 'expel',
 'the',
 'heads',
 'of',
 'two',
 'British',
 'aid',
 'agencies',
 '-',
 'Oxfam',
 'and',
 'Save',
 'the',
 'Children',
 '-',
 'citing',
 'administrative',
 'difficulties',
 'and',
 'humanitarian',
 'grounds',
 '.']

## 불용어

In [14]:
from nltk.corpus import stopwords
len( stopwords.words("english") )

179

## 품사 태깅
- 문장 속 단어에 품사를 붙이는 것

In [15]:
pos_df = pd.read_csv(f"{DATA_PATH}/data/pos_tag_list.csv")
pos_df

,Tag,Description
0,CC,coordinating conjuntion
1,CD,cardinal digit
2,DT,determiner
3,EX,existential there
4,FW,foreign word
5,IN,preposition/subordiateing conjunction
6,JJ,adjective
7,JJR,"adjective, comparative"
8,JJS,"adjective, superlative"
9,LS,list marker


In [16]:
words = word_tokenize(text.lower()) # 문장을 토큰화
words_pos = nltk.tag.pos_tag(words) # 토큰별 품사 태깅
words_pos

[('sudan', 'NN'),
 ('has', 'VBZ'),
 ('decided', 'VBN'),
 ('to', 'TO'),
 ('postpone', 'VB'),
 ('a', 'DT'),
 ('decision', 'NN'),
 ('to', 'TO'),
 ('expel', 'VB'),
 ('the', 'DT'),
 ('heads', 'NNS'),
 ('of', 'IN'),
 ('two', 'CD'),
 ('british', 'JJ'),
 ('aid', 'NN'),
 ('agencies', 'NNS'),
 ('-', ':'),
 ('oxfam', 'NN'),
 ('and', 'CC'),
 ('save', 'VB'),
 ('the', 'DT'),
 ('children', 'NNS'),
 ('-', ':'),
 ('citing', 'VBG'),
 ('administrative', 'JJ'),
 ('difficulties', 'NNS'),
 ('and', 'CC'),
 ('humanitarian', 'JJ'),
 ('grounds', 'NNS'),
 ('.', '.')]

- 명사나 동사만 학습데이터로 활용하고 싶다면???

```
N 으로 시작하면 명사
J 로 시작하면 형용사
R 로 시작하면 부사
V 로 시작하면 동사
```

In [17]:
[ token for token, pos in words_pos if pos.startswith("N") or pos.startswith("V") ]

['sudan',
 'has',
 'decided',
 'postpone',
 'decision',
 'expel',
 'heads',
 'aid',
 'agencies',
 'oxfam',
 'save',
 'children',
 'citing',
 'difficulties',
 'grounds']

```
문제.
N, V, J , R 로 시작하는 품사만 토큰화 해보세요.
```

In [18]:
[ token for token, pos in nltk.tag.pos_tag(words) if pos[0] in "NVJR" ]

['sudan',
 'has',
 'decided',
 'postpone',
 'decision',
 'expel',
 'heads',
 'british',
 'aid',
 'agencies',
 'oxfam',
 'save',
 'children',
 'citing',
 'administrative',
 'difficulties',
 'humanitarian',
 'grounds']

- 특수문자 제거와 소문자화 하기

In [19]:
train["clean"] = train["desc"].str.replace(r"[^\w ]+", "", regex=True).str.lower()
test["clean"] = test["desc"].str.replace(r"[^\w ]+", "", regex=True).str.lower()


- train["clean"] 의 모든 문서에 대해 불용어 제거 하고, 명사(N), 동사(V), 형용사(J), 부사(R) 만 토큰화 해서 train_list 라는 변수에 담아주세요.

In [20]:
train_list = []
stopwords_list = stopwords.words("english") # 불용어 리스트
for text in tqdm( train["clean"] ):
    # 토큰화
    tmp = word_tokenize(text)
    # 품사 태깅
    tmp = nltk.tag.pos_tag(tmp)
    # 불용어 제거와 명사(N), 동사(V), 형용사(J), 부사(R) 만 새로운 리스트에 담기
    token_list = [ token for token, pos in tmp if token not in stopwords_list and pos[0] in "NJVR" ]
    # train_list 에 추가!!
    train_list.append(token_list)

  0%|          | 0/89320 [00:00<?, ?it/s]

In [21]:
test_list = []
stopwords_list = stopwords.words("english") # 불용어 리스트
for text in tqdm( test["clean"] ):
    # 토큰화
    tmp = word_tokenize(text)
    # 품사 태깅
    tmp = nltk.tag.pos_tag(tmp)
    # 불용어 제거와 명사(N), 동사(V), 형용사(J), 부사(R) 만 새로운 리스트에 담기
    token_list = [ token for token, pos in tmp if token not in stopwords_list and pos[0] in "NJVR" ]
    # test_list 에 추가!!
    test_list.append(token_list)

  0%|          | 0/38280 [00:00<?, ?it/s]

In [22]:
len(train_list) , len(test_list)

(89320, 38280)

In [24]:
contain=[]
for con in train_list[:5]:
    contain.append(len(con))

contain # 패딩작업이 필요함

[17, 8, 13, 19, 21]

- 어휘집 만들기
    - 어휘집 만들고 어휘집의 총 단어개수를 저한테 DM으로 알려주세요.

In [25]:
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator(train_list, specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"]) # OOV 발생시 대체할 토큰 지정

c:\Users\hopio\anaconda3\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\hopio\anaconda3\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [26]:
len(vocab)

73685

- 각 토큰에 단어번호 부여하기

In [27]:
train_data = [ vocab(words)  for words in train_list ]
test_data = [ vocab(words)  for words in test_list ]

- 표제어 추출하기
    - 단어를 사전에 등재된 형태 변환

표제어(Lemma)는 언어학에서 단어의 기본 형태 또는 사전형을 의미합니다. **표제어 추출(Lemmatization)**은 단어의 변형된 형태들을 그 단어의 기본 형태로 변환하는 과정입니다. 예를 들어, 영어에서 "running", "ran", "runs" 같은 단어들은 모두 "run"이라는 표제어를 가집니다.

In [30]:
nltk.download("wordnet") # nltk 에서 표제어 추출하기 위해 다운로드

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hopio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [32]:
wnl.lemmatize("dogs")

'dog'

In [33]:
wnl.lemmatize("has")

'ha'

In [34]:
wnl.lemmatize("is")

'is'

In [35]:
wnl.lemmatize("is", "v") # v 는 동사, n 은 명사, a는 형용사, r 은 부사

'be'

# spacy
- 딥러닝 기반 형태소 분석 라이브러리

In [35]:
!pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.10-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.8-cp312-cp312-win_amd64.whl.metadata (8.6 kB)
  Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.5-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.4.8-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.4.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.2.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached blis-0.7.11-cp312-cp312-win_amd64.whl.metadata (7.6 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-

In [36]:
import spacy

In [38]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.9 MB/s eta 0:00:07
     ----- ---------------------------------- 1.8/12.8 MB 4.2 MB/s eta 0:00:03
     --------- ------------------------------ 3.1/12.8 MB 5.4 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 6.8 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 6.6 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.8 MB 7.7 MB/s eta 0:00:01
     ------------------------------------ --- 11.8/12.8 MB 8.3 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [37]:
nlp = spacy.load("en_core_web_sm") # 영어 형태소 분석 모델 불러오기

In [38]:
text = train["clean"][0]
text

'sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds'

In [39]:
doc = nlp(text)
doc

sudan has decided to postpone a decision to expel the heads of two british aid agencies  oxfam and save the children  citing administrative difficulties and humanitarian grounds

In [40]:
type(doc)

spacy.tokens.doc.Doc

In [41]:
len(doc) # 몇개의 토큰이 있는지 확인해보기

29

In [42]:
doc[28] # 토큰 객체

grounds

In [43]:
doc[0].text # 토큰 문자열

'sudan'

In [44]:
doc[1].lemma_ # 표제어

'have'

In [45]:
doc[1].tag_ # 품사 정보

'VBZ'

In [46]:
doc[1].is_alpha # 알파벳 여부

True

In [47]:
doc[1].is_stop # 불용어 여부

True

In [48]:
lst = [ [token.text, token.lemma_, token.tag_, token.is_alpha, token.is_stop ]  for token in doc ]
pd.DataFrame(lst, columns=["원래단어","표제어","품사","알파벳여부","불용어여부"])

,원래단어,표제어,품사,알파벳여부,불용어여부
0,sudan,sudan,NNP,True,False
1,has,have,VBZ,True,True
2,decided,decide,VBN,True,False
3,to,to,TO,True,True
4,postpone,postpone,VB,True,False
5,a,a,DT,True,True
6,decision,decision,NN,True,False
7,to,to,TO,True,True
8,expel,expel,VB,True,False
9,the,the,DT,True,True


- 불용어 제거와 함께 명사, 형용사, 부사, 동사만 토큰화 해보기

In [49]:
tmp_list = []
for text in tqdm( train["clean"][:100] ):
    doc = nlp(text) # 토큰 객체들을 담고 있는 doc 객체가 반환!
    tmp = [ token.text  for token in doc if not token.is_stop and token.tag_[0] in "NJVR" ]
    tmp_list.append(tmp)

  0%|          | 0/100 [00:00<?, ?it/s]

- 다음과 같이 하면 토큰화 속도는 빠르지만 표제어와 품사 정보가 포함이 되지 않는다.

In [54]:
nlp.tokenizer(text)

vienna iran and european negotiators reached a tentative compromise committing tehran to freezing a programme that can make nuclearweapons grade uranium but tehran still had to approve the agreement diplomats said last night

# padding

In [55]:
max_len = max( len(lst) for lst in train_data )
max_len

83

In [56]:
train_data = [ [0] * ( max_len - len(lst) ) + lst  if len(lst) < max_len else lst[:max_len]  for lst in train_data  ]
test_data = [ [0] * ( max_len - len(lst) ) + lst  if len(lst) < max_len else lst[:max_len]  for lst in test_data  ]

In [57]:
train_data = np.array(train_data)
test_data = np.array(test_data)

train_data.shape, test_data.shape

((89320, 83), (38280, 83))

In [58]:
target = train["target"].to_numpy()
target.shape, target.dtype

((89320,), dtype('int64'))

# 데이터셋 클래스

In [59]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y=None):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        item = {}
        item["x"] = torch.tensor(self.x[idx])
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item

In [60]:
dt = NewsDataset(train_data,target)
dl = torch.utils.data.DataLoader(dt, batch_size=2 , shuffle=False)
batch = next(iter(dl))
batch

{'x': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,   664,   723,  8069,   344,
          14527,  2664,    98,   712,  1590, 18705,  1124,   638,   903,  7190,
           7403,  3417,  5584],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
   

# 모델 클래스

In [61]:
class Net(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.emb_layer = torch.nn.Embedding(vocab_size, embedding_dim) # batch, seq, feature

        # 아래의 입력으로 batch, seq, feature -> batch, feature, seq 변환해서 전달
        self.conv1d_block = torch.nn.Sequential(
            torch.nn.Conv1d(embedding_dim, embedding_dim*2, 3), # batch, feature, seq
            torch.nn.ELU(),
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(embedding_dim*2, embedding_dim*4, 3), # batch, feature, seq
            torch.nn.ELU(),
            torch.nn.MaxPool1d(2),
        )
        self.rnn_layer = torch.nn.GRU(embedding_dim*4, embedding_dim*8, batch_first=True)

        self.output_layer = torch.nn.Linear(embedding_dim*8, 4) # output layer

    def forward(self, x):
        x = self.emb_layer(x)
        x = x.permute(0,2,1) # b, s, f -> b, f, s
        x = self.conv1d_block(x) # b , f, s
        x = x.permute(0,2,1) # b , f, s -> b , s, f
        _, hn = self.rnn_layer(x) # hn : nlayer, b, f
        return self.output_layer(hn[-1]) # (b, f)

In [62]:
model = Net( len(vocab), 64 )
model(batch["x"])

tensor([[-0.0056, -0.0906,  0.0039, -0.0422],
        [-0.0277, -0.0703,  0.0006, -0.0060]], grad_fn=<AddmmBackward0>)

# 학습 loop 함수

In [63]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in dataloader:
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

# 검증 및 테스트 loop 함수

In [98]:
inner_pred = []
inner_epoch_loss = []
inner_ep_cn = 0
inner_dt_cn = 0
dt_ln=0
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    global inner_ep_cn, inner_dt_cn,dt_ln  # global 키워드는 함수 시작 부분에서 선언
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Softmax(dim=1)
    model.eval()  # 평가 모드
    dt_ln=len(dataloader)

    for batch in dataloader:
        pred = model(batch["x"].to(device))

        if batch.get("y") is not None:
            loss = loss_fn(pred, batch["y"].to(device))

            print("="*5)
            print(f"오차 값: {loss.item()}") 
            
            # 배치(batch)로 여러 개의 샘플을 처리하지만, **손실 값(loss)**은 배치 전체에 대한 하나의 값으로 나옴
            # 배치 손실 계산: 배치 내 모든 샘플에 대해 계산한 손실 값을 평균하거나 합산하여 배치 전체의 손실 값

            print("="*5)

            epoch_loss += loss.item()
            inner_epoch_loss.append(epoch_loss)
            inner_ep_cn += 1  # 글로벌 변수 수정

        pred = act_func(pred)  # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy()  # cpu로 이동 후 ndarray로 변환
        inner_pred.append(pred)
        pred_list.append(pred)
        inner_dt_cn += 1  # 글로벌 변수 수정

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred


In [99]:
from sklearn.model_selection import KFold
n_=5
cv=KFold(n_,shuffle=True,random_state=SEED)

loss_fn = torch.nn.CrossEntropyLoss()

for i, (tri, vai) in enumerate( cv.split(train_data) ):
    # 학습용 데이터로더 객체
    train_dt = NewsDataset(train_data[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=2, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = NewsDataset(train_data[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=2, shuffle=False)


    if True:break

In [100]:
model = Net(len(vocab),2).to(device)
optimizer = torch.optim.Adam( model.parameters() )

test_loop(dl,model,loss_fn,device)

=====
오차 값: 1.3441882133483887
=====
=====
오차 값: 1.6462761163711548
=====
=====
오차 값: 1.2321226596832275
=====
=====
오차 값: 1.3240009546279907
=====
=====
오차 값: 1.501234769821167
=====
=====
오차 값: 1.2014474868774414
=====
=====
오차 값: 1.5111541748046875
=====
=====
오차 값: 1.6861534118652344
=====
=====
오차 값: 1.1799591779708862
=====
=====
오차 값: 1.1918361186981201
=====
=====
오차 값: 1.4733080863952637
=====
=====
오차 값: 1.6256471872329712
=====
=====
오차 값: 1.4907183647155762
=====
=====
오차 값: 1.6234508752822876
=====
=====
오차 값: 1.642310380935669
=====
=====
오차 값: 1.198893427848816
=====
=====
오차 값: 1.4860657453536987
=====
=====
오차 값: 1.310800552368164
=====
=====
오차 값: 1.5899416208267212
=====
=====
오차 값: 1.349757194519043
=====
=====
오차 값: 1.6124120950698853
=====
=====
오차 값: 1.0604103803634644
=====
=====
오차 값: 1.195241928100586
=====
=====
오차 값: 1.3364272117614746
=====
=====
오차 값: 1.5381073951721191
=====
=====
오차 값: 1.2228127717971802
=====
=====
오차 값: 1.5194015502929688
=====
=====
오

(1.4158831428904946,
 array([[0.34409875, 0.25627664, 0.19282818, 0.20679644],
        [0.3454658 , 0.25666776, 0.19759221, 0.20027426],
        [0.3577297 , 0.2541715 , 0.19088449, 0.19721432],
        ...,
        [0.34981573, 0.26141584, 0.19211641, 0.19665197],
        [0.36598662, 0.25881407, 0.18236211, 0.19283725],
        [0.33551455, 0.25147918, 0.19780982, 0.21519643]], dtype=float32))

In [89]:
inner_pred,len(inner_pred)

([array([[0.21643066, 0.23222063, 0.29658967, 0.25475904],
         [0.20845287, 0.23590676, 0.30161405, 0.2540263 ]], dtype=float32),
  array([[0.21451186, 0.2289707 , 0.30658543, 0.24993195],
         [0.21118146, 0.22183841, 0.31469882, 0.25228128]], dtype=float32),
  array([[0.20861264, 0.23061737, 0.30756247, 0.25320753],
         [0.2133651 , 0.2258003 , 0.3090035 , 0.25183114]], dtype=float32),
  array([[0.21225776, 0.22737136, 0.30405036, 0.2563206 ],
         [0.20443828, 0.23621622, 0.30540884, 0.25393665]], dtype=float32),
  array([[0.20982994, 0.2274762 , 0.3083668 , 0.25432706],
         [0.21299173, 0.23086049, 0.30804163, 0.2481061 ]], dtype=float32),
  array([[0.21573134, 0.2311594 , 0.30472887, 0.24838038],
         [0.2099042 , 0.22484142, 0.31121406, 0.25404036]], dtype=float32),
  array([[0.21398897, 0.2236596 , 0.30551934, 0.25683206],
         [0.20984668, 0.22838545, 0.30760512, 0.25416282]], dtype=float32),
  array([[0.20565057, 0.2181565 , 0.31623253, 0.2599603

In [87]:
inner_epoch_loss,len(inner_epoch_loss)

([1.3645460605621338,
  2.635899066925049,
  4.163589239120483,
  5.6239013671875,
  6.9451048374176025,
  8.45814561843872,
  9.796429753303528,
  11.071773409843445,
  12.57006585597992,
  14.082966804504395,
  15.49318790435791,
  16.78585147857666,
  18.22970449924469,
  19.48700249195099,
  20.676743030548096,
  22.196080446243286,
  23.51883101463318,
  24.971821188926697,
  26.24083924293518,
  27.610385060310364,
  28.99745810031891,
  30.557345986366272,
  32.083584666252136,
  33.45816993713379,
  34.777108788490295,
  36.287654995918274,
  37.62438607215881,
  38.87799644470215,
  40.1289381980896,
  41.64727854728699,
  43.10399031639099,
  44.273831605911255,
  45.5438494682312,
  46.800349950790405,
  48.301483511924744,
  49.59006154537201,
  50.96568691730499,
  52.24573540687561,
  53.7169646024704,
  54.981874108314514,
  56.44811296463013,
  57.77921211719513,
  59.196998715400696,
  60.63614773750305,
  62.074002742767334,
  63.41475510597229,
  64.76892113685608,
 

In [101]:
inner_ep_cn,inner_dt_cn,dt_ln

(44660, 44660, 44660)

# 하이퍼파라미터 정의

In [66]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=SEED)

batch_size = 32 # 배치 사이즈
loss_fn = torch.nn.CrossEntropyLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

vocab_size = len(vocab) # 단어 사전 크기
embedding_dim = 64 # 임베딩 벡터 크기

# 학습

In [67]:
os.getcwd()

'c:\\NLP_Papers_Review\\04_NLP'

In [69]:
is_holdout = False
reset_seeds(SEED) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_data) ):
    # 학습용 데이터로더 객체
    train_dt = NewsDataset(train_data[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = NewsDataset(train_data[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(vocab_size,embedding_dim).to(device)
    optimizer = torch.optim.Adam( model.parameters() )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = np.argmax(pred, axis=1) # 다중분류 문제에서 클래스 번호 결정
        score = f1_score(target[vai], pred, average="micro")

        #print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"{os.getcwd()}/data/news/weight/nltk_model_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 F1-Score micro: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

0번째 폴드 최고 F1-Score micro: 0.8725369458128078


  0%|          | 0/100 [00:00<?, ?it/s]

1번째 폴드 최고 F1-Score micro: 0.8751679355127631


  0%|          | 0/100 [00:00<?, ?it/s]

2번째 폴드 최고 F1-Score micro: 0.8749440214957457


  0%|          | 0/100 [00:00<?, ?it/s]

3번째 폴드 최고 F1-Score micro: 0.8757836990595611


  0%|          | 0/100 [00:00<?, ?it/s]

4번째 폴드 최고 F1-Score micro: 0.8714733542319749


In [70]:
np.mean(best_score_list)

0.8739811912225706